<a href="https://colab.research.google.com/github/rahulprasanth487/Semantic-Kernal-and-GraphRAG/blob/main/GraphRAG_Document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j
!pip install --upgrade --quiet  langchain-google-genai
!pip install --upgrade --quiet  langchain-openai langchain-experimental wikipedia tiktoken yfiles_jupyter_graphs

In [2]:

NEO4J_URI="neo4j+s://13f6c03c.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RZbRCl0pUoRTqPO0MoWYQN6PoMBdCbGUp_10H8Ws6bM"


In [3]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts.prompt import PromptTemplate

In [5]:
from google.colab import userdata
LLAMA_API_KEY=userdata.get('LLAMA_API_KEY')

In [6]:
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [7]:
import os

In [8]:
from langchain_community.vectorstores import Neo4jVector

In [9]:

os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [10]:
from langchain_community.graphs import Neo4jGraph

In [11]:
graph = Neo4jGraph()

In [12]:
graph

In [13]:
# from langchain.document_loaders import WikipediaLoader
# raw_documents = WikipediaLoader(query="Elizabeth I").load()

In [14]:
!pip install pypdf
!pip install rapidocr-onnxruntime

In [15]:
from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("/content/sample_data/Slice.pdf")
# raw_documents = loader.load_and_split()

In [26]:
loader = PyPDFLoader("/content/sample_data/Slice.pdf", extract_images=True)
raw_documents = loader.load()

In [27]:
len(raw_documents)

7

In [28]:
raw_documents

[Document(metadata={'source': '/content/sample_data/Slice.pdf', 'page': 0}, page_content='SLICE\nRajan\nBajaj\n-\nFounder\n&\nCEO\nat\nslice\nDeepak\nMalhotra\n-\nex-Co-Founder\nat\nslice\nYear\nof\nFoundation\n-\n2016\nWebsite\n-\nsliceit.com\nAbout:\n●Creditstartup-hasslefreepaymentcards-convertedtoEMI\n●Slicesupercards-Providezero-feecardstomaketransactionsinonlineandoffline\n●2%cashbackoneverytransactionusingSliceCreditcard\n●Creditlimit-2000to10Lakhs\n●NointerestifAmountrepaidwithin3EMIs\n●Thespendingsinthecardarealsoreflectedintheapp,likepatterns,repaymentreminders,etc.\nStartupStoryandTarget:\n●Themainreasonforthisstartupis,creditcardswereonesuchthingthatresultedinnumeroushasslesandplaguedthewholegeneration,especiallythosewhohailfrommiddle-classfamiliesinTier2andTier3cities.\n●TargetMarket:Studentsandearly-agebusinessassociateswhoareineligiblefortheusualcreditcard.\n●AverageageofSlicecustomers-22yrs.\nMission:\n●Thecompanyislookingtochangethisbyhelpingthestudentsgetcreditoptions

In [29]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [30]:
LLAMA_API_KEY
os.environ["LLAMA_API_KEY"] = LLAMA_API_KEY

In [31]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(api_key=userdata.get('GEMINI_API_KEY') ,model="gemini-pro",convert_system_message_to_human=True)
# response = llm.invoke("Explain Quantum Computing in 50 words?")
# print(response.content)

In [32]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=LLAMA_API_KEY,model_name="llama3-70b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f2eb2397e20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f2eb2395f30>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********'))

In [33]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [34]:
llm_transformer

In [35]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [36]:
graph_documents[0].nodes

[Node(id='Rajan Bajaj', type='Person'),
 Node(id='Deepak Malhotra', type='Person'),
 Node(id='Slice', type='Company')]

In [37]:
graph_documents[0].relationships

[Relationship(source=Node(id='Rajan Bajaj', type='Person'), target=Node(id='Slice', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Rajan Bajaj', type='Person'), target=Node(id='Slice', type='Company'), type='CEO'),
 Relationship(source=Node(id='Deepak Malhotra', type='Person'), target=Node(id='Slice', type='Company'), type='EX-CO-FOUNDER')]

In [38]:
graph_documents

[GraphDocument(nodes=[Node(id='Rajan Bajaj', type='Person'), Node(id='Deepak Malhotra', type='Person'), Node(id='Slice', type='Company')], relationships=[Relationship(source=Node(id='Rajan Bajaj', type='Person'), target=Node(id='Slice', type='Company'), type='FOUNDER'), Relationship(source=Node(id='Rajan Bajaj', type='Person'), target=Node(id='Slice', type='Company'), type='CEO'), Relationship(source=Node(id='Deepak Malhotra', type='Person'), target=Node(id='Slice', type='Company'), type='EX-CO-FOUNDER')], source=Document(metadata={'source': '/content/sample_data/Slice.pdf', 'page': 0}, page_content='SLICE\nRajan\nBajaj\n-\nFounder\n&\nCEO\nat\nslice\nDeepak\nMalhotra\n-\nex-Co-Founder\nat\nslice\nYear\nof\nFoundation\n-\n2016\nWebsite\n-\nsliceit.com\nAbout:\n●Creditstartup-hasslefreepaymentcards-convertedtoEMI\n●Slicesupercards-Providezero-feecardstomaketransactionsinonlineandoffline\n●2%cashbackoneverytransactionusingSliceCreditcard\n●Creditlimit-2000to10Lakhs\n●NointerestifAmountre

In [39]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [40]:
#directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [41]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [42]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [43]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [44]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [45]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.6 MB/s eta 0:00:00


In [46]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Neo4jVector

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

<ipython-input-46-4b2aa66d1228>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [47]:
vector_index

In [48]:
from langchain.chains import GraphCypherQAChain

In [49]:
# chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
# chain

In [50]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [51]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [53]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [54]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [57]:
entity_chain.invoke({"question": "who is the ceo of slice?"}).names

['slice']

In [58]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [59]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [60]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [62]:
print(structured_retriever("Who is the CEO of Slice?"))

Slice - PARTNERSHIP -> Amazon
Slice - PARTNERSHIP -> Flipkart
Slice - INVESTMENT -> Stride Ventures
Slice - INVESTMENT -> Tiger Global
Slice - INVESTMENT -> Moore Strategic Ventures
Slice - INVESTMENT -> Insight Partners
Slice - INVESTMENT -> Gunosy Capital
Slice - OWNS -> Tiger Global
Slice - OWNS -> Insight Partners
Slice - OWNS -> Moore Strategic
Rajan Bajaj - FOUNDER -> Slice
Rajan Bajaj - CEO -> Slice
Deepak Malhotra - EX-CO-FOUNDER -> Slice


In [63]:

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [64]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [65]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [66]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [67]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(groq_api_key=LLAMA_API_KEY,model_name="llama3-70b-8192")
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [68]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [69]:
prompt = ChatPromptTemplate.from_template(template)

In [70]:
# chain = (
#     RunnableParallel(
#         {
#             "context": _search_query | retriever,
#             "question": RunnablePassthrough(),
#         }
#     )
#     | prompt
#     | llm
#     | StrOutputParser()
# )

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | (lambda result: print(f"LLM Output: {result}") or result)  # Print LLM result
    | StrOutputParser()
)


In [71]:
chain.invoke({"question": "Who is Deebak?"})

Search query: Who is Deebak?


LLM Output: content='Deepak Malhotra is the ex-Co-Founder at Slice.' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 1302, 'total_tokens': 1317, 'completion_time': 0.04835758, 'prompt_time': 0.08074032, 'queue_time': 0.004884378999999994, 'total_time': 0.1290979}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None} id='run-f65b8ad7-0e31-4402-a5ff-569a9eefbe2f-0' usage_metadata={'input_tokens': 1302, 'output_tokens': 15, 'total_tokens': 1317}


'Deepak Malhotra is the ex-Co-Founder at Slice.'

In [74]:
chain.invoke(
    {
        "question": "What is the value of Slice?",
    }
)

Search query: What is the value of Slice?


LLM Output: content='The valuation of Slice is $1.8 billion as of March 2023.' response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1600, 'total_tokens': 1618, 'completion_time': 0.058071382, 'prompt_time': 0.124970257, 'queue_time': 0.010267441999999988, 'total_time': 0.183041639}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None} id='run-8faa6fcf-ae1c-42e3-9c63-ab3e1a0b4bd7-0' usage_metadata={'input_tokens': 1600, 'output_tokens': 18, 'total_tokens': 1618}


'The valuation of Slice is $1.8 billion as of March 2023.'

In [75]:
chain.invoke(
    {
        "question": "In which financial year Slice had a Huge profit and loss?",
    }
)

Search query: In which financial year Slice had a Huge profit and loss?


LLM Output: content='Slice did not have a huge profit in any financial year, but it had a significant loss in FY22, with a loss of Rs 253.67 crore.' response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 1519, 'total_tokens': 1553, 'completion_time': 0.109345531, 'prompt_time': 0.090939838, 'queue_time': 0.005076093000000004, 'total_time': 0.200285369}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None} id='run-13ef85ad-e025-45d8-b58c-d8cdc581bdff-0' usage_metadata={'input_tokens': 1519, 'output_tokens': 34, 'total_tokens': 1553}


'Slice did not have a huge profit in any financial year, but it had a significant loss in FY22, with a loss of Rs 253.67 crore.'